In [4]:
import findspark
from pyspark.sql import SparkSession
from pyspark.storagelevel import StorageLevel

findspark.init()
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [3]:
sc = spark.sparkContext

### Almacenamiento en caché

El almacenamiento en caché permite que Spark conserve los datos en todos los cálculos y operaciones.\
Es una de las técnicas más importantes de spark para acelerar los cálculos, especialmente cuando se trata de cálculos interactivos.\
Funciona almacenando el RDD tanto como sea posible en la memoria. Si los datos que se solicitan para almacenar en caché son más grandes que la memoria disponible, el rendimiento disminuirá porque se utilizará disco en lugar de memoria.

* Podemos marcar un RDD como almacenado en caché usando **persist()** o **cache()**
* **cache()** es un sinónimo de persist(MEMORY_ONLY)
* **persist()** puede usar memoria, disco o ambos

#### Valores posibles para el nivel de almacenamiento

* **MEMORY_ONLY** almacena el RDD como un objeto Java deserialización en la JVM. Si el RDD no cabe memoria, algunas particiones no se almacenarán en caché y se volverán a calcular sobre la marcha cada vez que se necesiten. Este es el nivel por defecto en spark.
* **MEORY_AND_DISK** almacena los RDD como objetos Java deserializados en la JVM. Si el RDD no cabe en memoria, almacena las particiones que no quepan en el disco y las lee desde allí cuando sea necesario.
* **DISK_ONLY** almacena las particiones del RD solo en disco.
* **MEMORY_ONLY_2, MEORY_AND_DISK_2, etc** igual que los niveles anteriores, pero replica cada partición en los nodos del cluster.

#### ¿Qué nivel de almacenamiento elegir?
El nivel de almacenamiento a elegir depende de la situación:

* Si los RDD caben en la memoria, use **MEMORY_ONLY**, ya que es la opción más rápida para el rendimiento de ejecución.
* **DISK_ONLY** no debe usarse a menos que sus cálculos sean costosos.
*  Utilice almacenamiento replicado para una mejor tolerancia fallos si puede ahorrar la memoria adicional necesaria. Esto evitará que se vuelvan a calcular las particiones perdidas para obtener la mejor disponibilidad.

>Podemos usar la función **unpersist()** para liberar el contenido en cahé

In [5]:
rdd = sc.parallelize([item for item in range(10)])

In [6]:
rdd.persist(StorageLevel.MEMORY_ONLY)

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:287

>\
>Si quisiéramos cambiar el nivel de persistencia al mismo RDD, debemos tener en cuenta que primero es necesario hacerle **unpersist()** a este mismo RDD. De lo contrario, si tratamos de cambiarle el nivel de persistencia sin realizar esta acción, nos va a devolver un error.
>
><br>

In [ ]:
rdd.unpersist()

rdd.persist(StorageLevel.DISK_ONLY)

In [ ]:
rdd.unpersist()
# Cuando le aplicamos cache(), estamos haciendo un MEMORY_ONLY
rdd.cache()

### Partition y shuffling (mezcla de datos)

* Los RDD operan con datos no como una sola masa de datos, sino que administran y operan los datos en particiones repartidas por todo el cluster. Por lo tanto, el concepto de partición de datos es fundamental para el correcto funcionamiento de los chop de Apache spark y puede tener un gran efecto en el rendimiento y en la forma en que se utilizan los recursos.

* Los RDD constan de particiones de datos y todas las operaciones se realizan en las particiones de datos en el RDD. Algunas operaciones, como las **transformaciones**, son funciones ejecutadas por un ejecutor en la partición específica de datos en la que se opera. Sin embargo, no todas las operaciones pueden realizarse simplemente realizando operaciones aisladas en las particiones de datos por parte de los respectivos ejecutores Las operaciones como las **agregaciones**, requieren que los datos se muevan a través del cluster en una fase conocida como mezcla o **shuffle**.

#### Importancia del número de particiones

* Si la cantidad de particiones es demasiado pequeña, usaremos sólo unas pocas CPU o núcleos en una gran cantidad de datos, por lo que tendremos un rendimiento más lento y dejaremos el cluster subutilizado.

* Si la cantidad de particiones es demasiado grande, utilizará más recursos de los que realmente necesita y en un entorno de múltiples procesos, podría estar provocando la falta de recursos para otros procesos que usted u otros miembros de su equipo ejecutan.

#### Particionadores

El particionamiento de los RDD se realiza mediante patinadores. Los patinadores asignan un índice de partición a los elementos del RDD. Todos los elementos de la misma partición tendrán el mismo índice de partición.\
Spark dispone de dos patinadores el HashPartitioner y el RangePartitioner.
Además de esto, también puede implementar un patrocinador personalizado.

* **HashPartitioner:** es el patrocinador predeterminado en spark y funciona calculando un valor hash para cada clave de los elementos. Todos los elementos con el mismo código hash terminan en una misma partición y esto se hace utilizando la fórmula siguiente:

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;partitionIndex = hash(key) % numPartitions

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;El partitionIndex es iguel al hash del elemento dividido entre el número de particiones